# Calculation of Metadata Amplification

In [3]:
import pandas as pd
import numpy as np

files = ["./performance-comparison-table-NOVA", "./performance-comparison-table-PMFS"] 
workloads = ["write", "randwrite", "fileserver.f", "varmail.f", "webserver.f", "webproxy.f"]

for fid, file in enumerate(files):
    with open(file, "r") as f:
        df = pd.read_csv(f, delim_whitespace=True, engine='python')

    for workload_idx, workload in enumerate(workloads):
        data_write = df[df['workloads'] == workload]["data_write(bytes)"].iloc[0]
        data_read = df[df['workloads'] == workload]["data_read(bytes)"].iloc[0]
        media_write = df[df['workloads'] == workload]["media_write(byte)"].iloc[0]
        media_read = df[df['workloads'] == workload]["media_read(byte)"].iloc[0]
        meta_write = df[df['workloads'] == workload]["meta_write(bytes)"].iloc[0]
        meta_read = df[df['workloads'] == workload]["meta_read(bytes)"].iloc[0]

        software_meta_IO = meta_write + meta_read
        real_meta_IO = 0 if media_write + media_read - media_write - data_write - data_read < 0 else media_write + media_read - media_write - data_write - data_read 

        print("workload: ", workload)
        print("software_meta_IO: ", software_meta_IO / 1024 / 1024 / 1024)
        print("real_meta_IO: ", real_meta_IO / 1024 / 1024 / 1024)


workload:  write
software_meta_IO:  5.118775148876011
real_meta_IO:  11.26172947883606
workload:  randwrite
software_meta_IO:  5.118775148876011
real_meta_IO:  12.526299238204956
workload:  fileserver.f
software_meta_IO:  4.144222942180932
real_meta_IO:  8.197546826675534
workload:  varmail.f
software_meta_IO:  4.509360040538013
real_meta_IO:  21.92910734936595
workload:  webserver.f
software_meta_IO:  0.63565603364259
real_meta_IO:  0.0
workload:  webproxy.f
software_meta_IO:  2.7977725798264146
real_meta_IO:  11.531339689157903
workload:  write
software_meta_IO:  2.8710703253746033
real_meta_IO:  9.23071002960205
workload:  randwrite
software_meta_IO:  2.8146859146654606
real_meta_IO:  13.51009464263916
workload:  fileserver.f
software_meta_IO:  4.967619827948511
real_meta_IO:  13.133158531971276
workload:  varmail.f
software_meta_IO:  30.896117341704667
real_meta_IO:  16.81608377210796
workload:  webserver.f
software_meta_IO:  1.301316793076694
real_meta_IO:  0.0
workload:  webproxy